In [3]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
#from sklearn import grid_search
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.linear_model import LogisticRegression
#from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier ,BaggingClassifier ,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import codecs
import os
from sklearn.svm import SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.neural_network import MLPClassifier

In [5]:
traindf = pd.read_json(codecs.open("train.json",'r','utf-8') )
traindf['ingredients_clean_string'] = [' , '.join(z).strip() for z in traindf['ingredients']]
traindf['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindf['ingredients']]       
testdf = pd.read_json(codecs.open("test.json",'r','utf-8') )
testdf['ingredients_clean_string'] = [' , '.join(z).strip() for z in testdf['ingredients']]
testdf['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in testdf['ingredients']]       

In [6]:
corpustr = traindf['ingredients_string']

In [7]:
estimators = [("tfidf", TfidfVectorizer(stop_words='english',
             ngram_range = ( 1 , 1 ),analyzer="word",
             max_df = .57 , binary=False ,max_features =6706, token_pattern=r'\w+' , sublinear_tf=False) ),
             ("hash", HashingVectorizer ( stop_words='english',
             ngram_range = ( 1 , 2 ),n_features  =6706,analyzer="word",token_pattern=r'\w+', binary =False))]

In [8]:
tfidftr = FeatureUnion(estimators).fit_transform(corpustr).todense()

In [9]:
corpusts = testdf['ingredients_string']

In [10]:
tfidfts = FeatureUnion(estimators).transform(corpusts)

In [11]:
predictors_tr = tfidftr

In [12]:
targets_tr = traindf['cuisine']

In [13]:
predictors_ts = tfidfts

In [14]:
classSVC = LinearSVC(C=0.3999, penalty="l2", dual=False) 

In [15]:
classSVC = classSVC.fit(predictors_tr,targets_tr)

C:\Users\behlul\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [16]:
predictions = classSVC.predict(predictors_ts) 

In [19]:
testdf['cuisine'] = predictions
testdf = testdf.sort_values('id' , ascending=True)

In [20]:
testdf[['id' , 'cuisine' ]].to_csv("final submission.csv", index=False)